In [48]:
import pandas as pd

md = pd.read_csv('mangadex_optimized.csv')
mw = pd.read_csv('mangaworld_matched.csv')
ma = pd.read_csv('myanimelist_fixed_full.csv')


In [49]:

keep_md = ['manga_id', 'title_api', 'follows', 'rating', 'content_rating', 'demographic', 'status', 'year', 'last_updated']
md = md[keep_md]

keep_mw = ['manga_id', 'title', 'views', 'type', 'status', 'year']
mw = mw[keep_mw]

keep_ma = ['title','manga_id', 'type', 'source', 'status','aired_from','aired_to','studios','genres', 'year', 'rating', 'score', 'scored_by', 'popularity', 'rank', 'members', 'favorites', 'genres', 'themes', 'demographics']
ma = ma[keep_ma]

In [50]:
md_map  = md.groupby('manga_id').first()
mw_map  = mw.groupby('manga_id').first()
ma_map  = ma.groupby('manga_id').first()

In [51]:
for col in ['year', 'status']:
    md[col] = md[col].fillna(md['manga_id'].map(mw_map[col]))
    md[col] = md[col].fillna(md['manga_id'].map(ma_map[col]))
md['rating']      = md['rating'].fillna(md['manga_id'].map(ma_map['score']))
md['demographic'] = md['demographic'].fillna(md['manga_id'].map(ma_map['demographics']))

for col in ['year', 'status']:
    mw[col] = mw[col].fillna(mw['manga_id'].map(md_map[col]))
    mw[col] = mw[col].fillna(mw['manga_id'].map(ma_map[col]))
mw['type'] = mw['type'].fillna(mw['manga_id'].map(ma_map['type']))


ma['status'] = ma['status'].fillna(ma['manga_id'].map(md_map['status']))
ma['status'] = ma['status'].fillna(ma['manga_id'].map(mw_map['status']))
ma['type'] = ma['type'].fillna(ma['manga_id'].map(mw_map['type']))
ma['demographics'] = ma['demographics'].fillna(ma['manga_id'].map(md_map['demographic']))



In [52]:
ma['aired_from']=pd.to_datetime(ma['aired_from'],dayfirst=True,errors='coerce')
ma['aired_to']=pd.to_datetime(ma['aired_to'],dayfirst=True,errors='coerce')
ma['aired_to_year']=ma['aired_to'].dt.year
ongoing_ids=set(md.loc[md['status']=='ongoing','manga_id'])
mask=ma['manga_id'].isin(ongoing_ids)&(ma['aired_to_year']>=2024)
ma.loc[mask,'status']='Currently Airing'
ma.drop(columns=['aired_to_year'],inplace=True)

In [53]:
md.to_csv('mangadex.csv', index=False)
mw.to_csv('mangaworld.csv', index=False)
ma.to_csv('myanimelist.csv', index=False)